# Feature engineering on sensor data - how to overcome feature explosion

The purpose of this notebook is to illustrate how we can overcome the feature explosion problem based on an example dataset involving sensor data.


Summary:

- Prediction type: __Regression__
- Domain: __Robotics__
- Prediction target: __The force vector on the robot's arm__ 
- Population size: __15001__

_Author: Dr. Patrick Urbanke_

## The data set

The data set has been generously provided by Erik Berger who originally collected it for his dissertation:

> Berger, E. (2018). *Behavior-Specific Proprioception Models for Robotic Force Estimation: A Machine Learning Approach.* Freiberg, Germany: Technische Universitaet Bergakademie Freiberg.

## A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

## 1. Loading data

We begin by importing the libraries and setting the project.

In [11]:
import datetime
import os
from urllib import request
import time

import pandas as pd
import numpy as np
import getml
import getml.data as data
import getml.database as database
import getml.engine as engine
import getml.feature_learning.aggregations as agg
import getml.data.roles as roles

from utils import FTTimeSeriesBuilder, TSFreshBuilder

from IPython.display import Image
import matplotlib.pyplot as plt
plt.style.use('seaborn')
%matplotlib inline  
 
getml.engine.set_project('robot')


Connected to project 'robot'


### 1.1 Download from source


In [12]:
fname = "robot-demo.csv"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://static.getml.com/datasets/robotarm/" + fname, 
        fname
    )

In [13]:
data_all = getml.data.DataFrame.from_csv("robot-demo.csv", "data_all")

In [14]:
data_all

Name              3              4              5              6   ...            105            106            f_x
 Role   unused_float   unused_float   unused_float   unused_float   ...   unused_float   unused_float   unused_float
    0         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -11.03 
    1         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.848
    2         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.666
    3         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.507
    4         3.4098        -0.3274         0.9604        -3.7436   ...         47.955         47.971        -10.413
             ...            ...            ...            ...                   ...            ...            ...   
14996         3.0837        -0.8836         1.4501        -2.2102   ...         47.94          47.94          10.84 
14997         3.0835        -0.884          1.4505        -2.2091   ...         47.94          47.94          10.857
14998         3.0833        -0.8844         1.4508        -2.208    ...         47.94          47.94          10.89 
14999         3.0831        -0.8847         1.4511        -2.2071   ...         47.94          47.94          11.29 
15000         3.0829        -0.885          1.4514        -2.2062   ...         47.94          47.955         11.69 

 Name            f_y            f_z
 Role   unused_float   unused_float
    0         6.9           -7.33  
    1         6.7218        -7.4427
    2         6.5436        -7.5555
    3         6.4533        -7.65  
    4         6.6267        -7.69  
             ...            ...    
14996        -1.41          16.14  
14997        -1.52          15.943 
14998        -1.74          15.55  
14999        -1.4601        15.743 
15000        -1.1801        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
name: data_all
type: getml.data.DataFrame
url: http://localhost:1709/#/getdataframe/robot/data_all/

### 1.2 Prepare data

The force vector consists of three component (*f_x*, *f_y* and *f_z*), meaning that we have three targets. For this comparison, we only predict the first component (*f_x*). 

Also, we want to speed things up a little, so we only use 10 columns. A previous analysis has revealed that the predictive power is mainly extracted from these 10 columns.

In [15]:
only_use = ['30', '34', '37', '38', '4', '59', '61', '7', '77', '78']

data_all.set_role(["f_x"], getml.data.roles.target)
data_all.set_role(only_use, getml.data.roles.numerical)

This is what the data set looks like:

In [16]:
data_all

Name       f_x          30          34          37   ...            104            105            106            f_y
 Role    target   numerical   numerical   numerical   ...   unused_float   unused_float   unused_float   unused_float
    0   -11.03      -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.9   
    1   -10.848     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.7218
    2   -10.666     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.5436
    3   -10.507     -1.2042     -0.3273     -1.0191   ...         47.834         47.955         47.971         6.4533
    4   -10.413     -1.2042     -0.3274     -1.0191   ...         47.834         47.955         47.971         6.6267
         ...        ...         ...         ...                   ...            ...            ...           ...    
14996    10.84      -2.6343     -0.8836     -1.5591   ...         47.803         47.94          47.94         -1.41  
14997    10.857     -2.6331     -0.8841     -1.5596   ...         47.803         47.94          47.94         -1.52  
14998    10.89      -2.6321     -0.8844     -1.56     ...         47.803         47.94          47.94         -1.74  
14999    11.29      -2.6311     -0.8848     -1.5604   ...         47.803         47.94          47.94         -1.4601
15000    11.69      -2.6302     -0.8851     -1.5607   ...         47.803         47.94          47.955        -1.1801

 Name            f_z
 Role   unused_float
    0        -7.33  
    1        -7.4427
    2        -7.5555
    3        -7.65  
    4        -7.69  
             ...    
14996        16.14  
14997        15.943 
14998        15.55  
14999        15.743 
15000        15.937 


15001 rows x 96 columns
memory usage: 11.52 MB
name: data_all
type: getml.data.DataFrame
url: http://localhost:1709/#/getdataframe/robot/data_all/

### 1.3 Separate data into a training and testing set

We also want to separate the data set into a training and testing set. We do so by using the first 10,500 measurements for training and then using the remainder for testing.

In [17]:
separator = 10500

data_train = data_all.where("data_train", data_all.rowid() < separator)
data_test = data_all.where("data_test", data_all.rowid() >= separator)

## 2. Predictive modeling

### 2.1 Propositionalization with getML's FastProp

In [18]:
fast_prop = getml.feature_learning.FastPropTimeSeries(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    allow_lagged_targets=False,
    memory=15,
    num_threads=1,
)


In [19]:
pipe_fp_fl = getml.pipeline.Pipeline(
    feature_learners=[fast_prop],
    tags=["feature learning", "fastprop"],
)

In [20]:
pipe_fp_fl.check(data_train)

Checking data model...
OK.


In [21]:
begin = time.time()

pipe_fp_fl.fit(data_train)

fastprop_train = pipe_fp_fl.transform(data_train, df_name="fastprop_train")

end = time.time()

fastprop_runtime = datetime.timedelta(seconds=end - begin)

Checking data model...
OK.

FastProp: Trying 123 features...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:0.064701


FastProp: Building features...
[========================================] 100%



In [22]:
fastprop_test = pipe_fp_fl.transform(data_test, df_name="fastprop_test")


FastProp: Building features...
[========================================] 100%



In [23]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [24]:
pipe_fp_pr.check(fastprop_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_113' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_115' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_118' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_119' in POPULATION__STAGING_T

Checking data model...


In [25]:
pipe_fp_pr.fit(fastprop_train)

WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_113' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_115' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_118' in POPULATION__STAGING_TABLE_1 are equal to each other. You should consider setting its role to unused_float or using it for comparison only (you can do the latter by setting a unit that contains 'comparison only').
WARNING [COLUMN SHOULD BE UNUSED]: All non-NULL entries in column 'feature_1_119' in POPULATION__STAGING_T

Checking data model...

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.419197



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'fastprop'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/robot/dCBBdf/0/

In [26]:
pipe_fp_pr.score(fastprop_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-18 13:17:28,fastprop_train,f_x,0.4479,0.5908,0.9961
1,2021-05-18 13:17:29,fastprop_test,f_x,0.5593,0.7332,0.9949


### 2.2 Propositionalization with featuretools

In [27]:
dfs_pandas = {}

for df in [data_train, data_test, data_all]:
    dfs_pandas[df.name] = df.to_pandas()
    delete_columns = [col for col in dfs_pandas[df.name].columns if col not in only_use + ["f_x"]]
    for col in delete_columns:
        del dfs_pandas[df.name][col] 
    dfs_pandas[df.name]["id"] = 1
    dfs_pandas[df.name]["ds"] = pd.to_datetime(np.arange(0, dfs_pandas[df.name].shape[0]), unit="s")

In [28]:
dfs_pandas["data_train"]

,30,34,37,38,4,59,61,7,77,78,f_x,id,ds
0,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082791,0.78597,-1.0191,0.082782,-1.4094,-11.0300,1,1970-01-01 00:00:00
1,-1.2042,-0.32739,-1.0191,-6.0205,-0.32737,0.082800,0.78592,-1.0191,0.082782,-1.4094,-10.8480,1,1970-01-01 00:00:01
2,-1.2042,-0.32737,-1.0191,-6.0205,-0.32737,0.082786,0.78594,-1.0191,0.082782,-1.4094,-10.6660,1,1970-01-01 00:00:02
3,-1.2042,-0.32734,-1.0191,-6.0205,-0.32737,0.082755,0.78599,-1.0191,0.082782,-1.4094,-10.5070,1,1970-01-01 00:00:03
4,-1.2042,-0.32736,-1.0191,-6.0205,-0.32737,0.082782,0.78597,-1.0191,0.082782,-1.4094,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.1446,-0.37311,-1.0486,-5.9532,-0.37326,0.087343,0.90793,-1.0488,0.087468,-1.4162,-9.7673,1,1970-01-01 02:54:55
10496,-1.1349,-0.37103,-1.0472,-5.9564,-0.37108,0.087241,0.90199,-1.0474,0.087274,-1.4160,-9.9200,1,1970-01-01 02:54:56
10497,-1.1255,-0.36889,-1.0458,-5.9596,-0.36896,0.087055,0.89618,-1.0460,0.087082,-1.4158,-9.7743,1,1970-01-01 02:54:57
10498,-1.1163,-0.36680,-1.0444,-5.9627,-0.36689,0.086907,0.89034,-1.0447,0.086893,-1.4155,-8.6109,1,1970-01-01 02:54:58


In [29]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(seconds=0),
    memory=pd.Timedelta(seconds=15),
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [30]:
featuretools_train = ft_builder.fit(dfs_pandas["data_train"])
featuretools_test = ft_builder.transform(dfs_pandas["data_test"])

featuretools: Trying features...


/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 177 features...
Time taken: 0h:6m:8.647821



/Users/srnnkls/getML/monorepo/src/python-api/.venv/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [31]:
featuretools_train

,MIN(peripheral.37),MIN(peripheral.7),FIRST(peripheral.37),FIRST(peripheral.7),MEDIAN(peripheral.37),MEDIAN(peripheral.7),MEAN(peripheral.37),MEAN(peripheral.7),SUM(peripheral.37),SUM(peripheral.7),...,"TREND(peripheral.38, ds)","TREND(peripheral.59, ds)","TREND(peripheral.30, ds)","TREND(peripheral.4, ds)","TREND(peripheral.78, ds)","TREND(peripheral.34, ds)","TREND(peripheral.77, ds)",f_x,id,ds
_featuretools_index,,,,,,,,,,,,,,,,,,,,,
0,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-1.0191,-1.0191,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-11.0300,1,1970-01-01 00:00:00
1,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-2.0382,-2.0382,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-10.8480,1,1970-01-01 00:00:01
2,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-3.0573,-3.0573,...,-1.204867e-26,-0.216000,0.000000,0.000000,0.000000,0.864000,0.000000,-10.6660,1,1970-01-01 00:00:02
3,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-4.0764,-4.0764,...,0.000000e+00,-1.054080,0.000000,0.000000,0.000000,1.468800,0.000000,-10.5070,1,1970-01-01 00:00:03
4,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.0191,-1.019100,-1.019100,-5.0955,-5.0955,...,0.000000e+00,-0.544320,0.000000,0.000000,0.000000,0.950400,0.000000,-10.4130,1,1970-01-01 00:00:04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,-1.0716,-1.0721,-1.0716,-1.0721,-1.0596,-1.0596,-1.059793,-1.059927,-15.8969,-15.8989,...,-3.789643e-03,-0.000183,0.011060,0.002575,0.000191,0.002561,-0.000201,-9.7673,1,1970-01-01 02:54:55
10496,-1.0698,-1.0702,-1.0698,-1.0702,-1.0580,-1.0580,-1.058167,-1.058280,-15.8725,-15.8742,...,-3.708571e-03,-0.000179,0.010880,0.002522,0.000198,0.002506,-0.000201,-9.9200,1,1970-01-01 02:54:56
10497,-1.0681,-1.0684,-1.0681,-1.0684,-1.0565,-1.0563,-1.056567,-1.056667,-15.8485,-15.8500,...,-3.629286e-03,-0.000177,0.010697,0.002469,0.000206,0.002450,-0.000200,-9.7743,1,1970-01-01 02:54:57


In [32]:
roles={
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=roles
)


In [33]:
df_featuretools_train.set_role(
    df_featuretools_train.unused_names, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.unused_names, getml.data.roles.numerical
)

In [34]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

In [35]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.722218



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['prediction', 'featuretools'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/robot/t1gwxU/0/

In [36]:
pipe_ft_pr.score(df_featuretools_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-18 13:26:23,featuretools_train,f_x,0.4309,0.5701,0.9964
1,2021-05-18 13:26:23,featuretools_test,f_x,0.5596,0.7385,0.9948


### 2.3 Propositionalization with tsfresh

In [37]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, 
    memory=15, 
    column_id="id",
    time_stamp="ds",
    target="f_x",
)

In [38]:
tsfresh_train = tsfresh_builder.fit(dfs_pandas["data_train"])

tsfresh_test = tsfresh_builder.transform(dfs_pandas["data_test"])

Feature Extraction: 100%|██████████| 20/20 [00:37<00:00,  1.87s/it]


Selecting the best out of 120 features...
Time taken: 0h:1m:19.589981



Feature Extraction: 100%|██████████| 20/20 [00:16<00:00,  1.19it/s]


In [39]:
roles={
    getml.data.roles.target: ["f_x"],
    getml.data.roles.join_key: ["id"],
    getml.data.roles.time_stamp: ["ds"],
}

df_tsfresh_train = getml.data.DataFrame.from_pandas(
    tsfresh_train, name="tsfresh_train", roles=roles
)

df_tsfresh_test = getml.data.DataFrame.from_pandas(
    tsfresh_test, name="tsfresh_test", roles=roles
)

In [40]:
df_tsfresh_train.set_role(df_tsfresh_train.unused_names, getml.data.roles.numerical)

df_tsfresh_test.set_role(df_tsfresh_test.unused_names, getml.data.roles.numerical)

In [41]:
pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["predicition", "tsfresh"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

In [42]:
pipe_tsf_pr.check(df_tsfresh_train)

Checking data model...
OK.


In [43]:
pipe_tsf_pr.fit(df_tsfresh_train)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:4.324484



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['predicition', 'tsfresh'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/robot/Jx8SzV/0/

In [44]:
pipe_tsf_pr.score(df_tsfresh_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-18 13:28:25,tsfresh_train,f_x,0.4873,0.6564,0.9952
1,2021-05-18 13:28:25,tsfresh_test,f_x,0.5939,0.7857,0.9939


### 3. Comparison

In [45]:
num_features = dict(
    fastprop=123,
    featuretools=177,
    tsfresh=110,
)

runtime_per_feature = [
    fastprop_runtime / num_features['fastprop'],
    ft_builder.runtime / num_features['featuretools'],
    tsfresh_builder.runtime / num_features['tsfresh'],
]

features_per_second = [1.0/r.total_seconds() for r in runtime_per_feature]

speedup_per_feature = [r/runtime_per_feature[0] for r in runtime_per_feature]

comparison = pd.DataFrame(
    dict(
        runtime=[fastprop_runtime, ft_builder.runtime, tsfresh_builder.runtime],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        speedup=[1, ft_builder.runtime/fastprop_runtime, tsfresh_builder.runtime/fastprop_runtime],
        speedup_per_feature=speedup_per_feature,
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

In [46]:
comparison

,runtime,num_features,features_per_second,speedup,speedup_per_feature,mae,rmse,rsquared
getML: FastProp,0 days 00:00:03.929530,123,31.301844,1.000000,1.000000,0.559264,0.733232,0.994929
featuretools,0 days 00:06:08.647821,177,0.480133,93.814736,65.194103,0.559580,0.738467,0.994774
tsfresh,0 days 00:01:19.589981,110,1.382084,20.254326,22.648292,0.593878,0.785746,0.993929


In [47]:
comparison.to_csv("comparisons/robot.csv")

## 3. Conclusion


The purpose of this notebook has been to illustrate the problem of the curse of dimensionality when engineering features from datasets with many columns.

The most important thing to remember is that this problem exists regardless of whether you engineer your features manually or using algorithms. Whether you like it or not: If you write your features in the traditional way, your search space grows quadratically with the number of columns.

# Next Steps

This tutorial explained how to overcome the problem of feature explosion with getML's feature learning algorithms Multirel and Relboost. 

If you are interested in further real-world applications of getML, head back to the [notebook overview](welcome.md) and choose one of the remaining examples. 

Here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Feature learning with Multirel](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#multirel)
* [Feature learning with Relboost](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#relboost)

# Get in contact

If you have any question schedule a [call with Alex](https://go.getml.com/meetings/alexander-uhlig/getml-demo), the co-founder of getML, or write us an [email](team@getml.com). Prefer a private demo of getML? Just contact us to make an appointment.